In [16]:
!pip install ipython-autotime

time: 3.18 s (started: 2021-09-04 23:20:44 +00:00)


In [17]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import models,transforms,datasets

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.51 ms (started: 2021-09-04 23:20:47 +00:00)


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

time: 9.42 ms (started: 2021-09-04 23:20:47 +00:00)


# Import CIFAR-100 and Resize

In [19]:
train_transform = transforms.Compose([transforms.Resize(64),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.RandomRotation(15),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))])

test_transform = transforms.Compose([transforms.Resize(64),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))])

time: 6.3 ms (started: 2021-09-04 23:20:47 +00:00)


In [20]:
train_set = datasets.CIFAR100(root = "CIFAR100", train = True, download = True, transform = train_transform)

Files already downloaded and verified
time: 993 ms (started: 2021-09-04 23:20:47 +00:00)


In [21]:
test_set = datasets.CIFAR100(root = "CIFAR100", train = False, download = True, transform = test_transform)

Files already downloaded and verified
time: 839 ms (started: 2021-09-04 23:20:48 +00:00)


In [22]:
train_set[0][0].shape

torch.Size([3, 64, 64])

time: 9.68 ms (started: 2021-09-04 23:20:49 +00:00)


In [23]:
test_set[0][0].shape

torch.Size([3, 64, 64])

time: 6.44 ms (started: 2021-09-04 23:20:49 +00:00)


# VGG16

In [24]:
# Number of classes in the dataset
num_classes = 100

# Batch size for training
batch_size = 64

# Number of epochs
num_epochs = 15

time: 2.65 ms (started: 2021-09-04 23:20:49 +00:00)


In [25]:
vgg16_model = models.vgg16(pretrained = True) #130million+ parameters

#set_parameter_requires_grad for classifier layers
'''
for param in vgg16_model.parameters():
  param.requires_grad=False
'''

#Modify last layer out classes
vgg16_model.classifier[6].out_features = num_classes

vgg16_model = vgg16_model.to(device)

time: 1.76 s (started: 2021-09-04 23:20:49 +00:00)


In [26]:
print(vgg16_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Training

In [27]:
criterion = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(vgg16_model.parameters(), lr = 0.003, momentum = 0.9)
lr_scheduler = optim.lr_scheduler.StepLR(sgd_optimizer, step_size = 15, gamma = 0.1)

train_dataloader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_set, batch_size = batch_size, shuffle = True)

time: 4.63 ms (started: 2021-09-04 23:20:51 +00:00)


In [28]:
def fit(model, dataset, optimizer, scheduler, criterion):
  #Set model to training mode
  model.train()
  scheduler.step()
  #Iterate over data
  for data, targets in dataset:
    data = data.to(device)
    targets = targets.to(device)
    #Reset the gradients
    optimizer.zero_grad()
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()

time: 8.25 ms (started: 2021-09-04 23:20:51 +00:00)


In [29]:
def validate(model, dataset, criterion):
  #set model to evaluation mode
  model.eval()
  acc = 0
  loss = []
  for data, targets in dataset:
    data = data.to(device)
    targets = targets.to(device)
    out = model(data)
    #Get loss
    l = criterion(out, targets)
    loss.append(l.item())
    #Get index of class label
    _,preds = torch.max(out.data,1)
    #Get accuracy
    acc += torch.sum(preds == targets).item()

  return 100*acc/len(dataset.dataset), np.mean(np.array(loss))

time: 11.5 ms (started: 2021-09-04 23:20:51 +00:00)


In [30]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []

#Loop for num_epochs
for epoch in range(num_epochs):
  #Fit model
  fit(vgg16_model, train_dataloader, sgd_optimizer, lr_scheduler, criterion)

  #Train validation
  trn_acc, trn_lss = validate(vgg16_model, train_dataloader, criterion)
  train_loss.append(trn_lss)
  train_acc.append(trn_acc)

  #Test validation
  tst_acc, tst_lss = validate(vgg16_model, test_dataloader, criterion)
  test_loss.append(tst_lss)
  test_acc.append(tst_acc)

  print(f'Epoch:{epoch+1:2.0f}\t Train_Loss:{train_loss[-1]:.4f}\t Train_Acc:{train_acc[-1]:.4f}\t Test_Loss:{test_loss[-1]:.4f}\t Test_Acc:{test_acc[-1]:.4f}')


Epoch: 1	 Train_Loss:2.6165	 Train_Acc:29.1160	 Test_Loss:2.6036	 Test_Acc:29.8900
Epoch: 2	 Train_Loss:1.7860	 Train_Acc:49.4560	 Test_Loss:1.8008	 Test_Acc:49.0100
Epoch: 3	 Train_Loss:1.3836	 Train_Acc:60.4160	 Test_Loss:1.4705	 Test_Acc:57.4100
Epoch: 4	 Train_Loss:1.1918	 Train_Acc:64.6220	 Test_Loss:1.3559	 Test_Acc:60.7700
Epoch: 5	 Train_Loss:1.0223	 Train_Acc:70.1380	 Test_Loss:1.2664	 Test_Acc:64.4300
Epoch: 6	 Train_Loss:0.8891	 Train_Acc:73.6700	 Test_Loss:1.1874	 Test_Acc:66.1600
Epoch: 7	 Train_Loss:0.7656	 Train_Acc:76.9820	 Test_Loss:1.1402	 Test_Acc:67.9100
Epoch: 8	 Train_Loss:0.6762	 Train_Acc:79.5380	 Test_Loss:1.1100	 Test_Acc:69.2200
Epoch: 9	 Train_Loss:0.6017	 Train_Acc:81.6420	 Test_Loss:1.0874	 Test_Acc:70.3600
Epoch:10	 Train_Loss:0.5797	 Train_Acc:82.6300	 Test_Loss:1.0845	 Test_Acc:70.4200
Epoch:11	 Train_Loss:0.5090	 Train_Acc:84.2100	 Test_Loss:1.1006	 Test_Acc:70.1500
Epoch:12	 Train_Loss:0.4783	 Train_Acc:85.2940	 Test_Loss:1.1588	 Test_Acc:70.1600
Epoc

# Save Model

In [31]:
torch.save(vgg16_model.state_dict(), 'VGG16(SGD)')

time: 1.87 s (started: 2021-09-05 00:40:35 +00:00)
